In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

import time
from datetime import datetime
from fake_useragent import UserAgent
import random

import sqlite3
import re

import pandas as pd

from nltk.corpus import stopwords
from pymystem3 import Mystem

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
def pars_get_id(article_code):
    post = article_code.find_all('a', {'class': 'postbtn-reply-href'})
    return int(post[0].get_text())

In [2]:
def pars_get_title(article_code):
    post = article_code.find('title')
    if post is not None:
        return post.get_text()
    else:
        return None

In [20]:
def pars_get_date(article_code):
    post = article_code.find_all('span', {'class': 'posttime'})
    return post[0].get_text()

In [13]:
def pars_get_text(article_code):
    post = article_code.find_all('blockquote', {'class': 'post-message'})
    return post[0].get_text()

In [70]:
def write_to_db_data(data):
    cur.execute(
    """
    INSERT INTO threads 
        (id, text, name, date, link)
        VALUES (?, ?, ?, ?, ?)
    """, data)
    conn.commit()
    return None

In [65]:
def write_to_db_sections(id_sec, section):
    cur.execute(
    """
    INSERT INTO sections 
        (id_section, name_section)
        VALUES (?, ?)
    """, (id_sec, section)) 
    conn.commit()
    return None

In [66]:
def write_to_db_conn_section(id_sec, id_thread):
    cur.execute(
    """
    INSERT INTO conn_section 
        (id, id_section)
        VALUES (?, ?)
    """, (id_thread, id_sec)) 
    conn.commit()
    return None

In [12]:
def write_error(error):
    with open('errores.txt', 'a') as f:
        f.write(str(error) + '\n')
    return None

In [58]:
conn = sqlite3.connect('dvach_threads.db')
cur = conn.cursor()

In [69]:
cur.execute("""
CREATE TABLE IF NOT EXISTS threads 
(id INTEGER PRIMARY KEY, text text, name text, date text, link text)
""")
conn.commit()

In [60]:
cur.execute("""
CREATE TABLE IF NOT EXISTS sections 
(id_section INTEGER PRIMARY KEY, name_section text)
""")
conn.commit()

In [61]:
cur.execute("""
CREATE TABLE IF NOT EXISTS conn_section 
(id INTEGER NOT NULL, id_section INTEGER NOT NULL, FOREIGN KEY (id) REFERENCES threads(id), FOREIGN KEY (id_section) REFERENCES sections(id_section))
""")
conn.commit()

In [5]:
session = requests.session()

In [6]:
adapter = requests.adapters.HTTPAdapter()

In [7]:
session.mount('https://', adapter)

In [8]:
ua = UserAgent(verify_ssl=False)

In [9]:
headers = {'User-Agent': ua.random}

In [10]:
known_proxy_ip = '194.158.203.14:80'

In [11]:
proxy = {'http': known_proxy_ip}

In [41]:
falselinks = []
for i in range(1753):
    falselinks.append(str(i))
falselinks.append('Два.ч')

In [50]:
b_links = []

In [76]:
b_links[5495]

'/b/arch/2016-03-11/res/119624956.html'

In [82]:
link = 'https://2ch.hk/b/arch/2016-03-11/res/119624956.html'
response = session.get(link, headers=headers, proxies=proxy)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

True

In [52]:
for idn in range(0, 101):
    try:
        response = session.get('https://2ch.hk/b/arch/' + str(idn) + '.html', headers=headers, proxies=proxy)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        write_error(e)
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    post = soup.find_all('a', href=True)
    for i in post:
        if i.get_text() not in falselinks:
            b_links.append(i['href'])
    
    print(datetime.now())
    time.sleep(random.uniform(1.1, 2.2))

2023-03-26 15:34:34.142623
2023-03-26 15:34:35.932384
2023-03-26 15:34:37.636150
2023-03-26 15:34:40.216939
2023-03-26 15:34:42.704812
2023-03-26 15:34:44.618505
2023-03-26 15:34:47.164292
2023-03-26 15:34:48.808518
2023-03-26 15:34:50.700398
2023-03-26 15:34:53.081800
2023-03-26 15:34:55.566169
2023-03-26 15:34:58.254995
2023-03-26 15:35:00.763993
2023-03-26 15:35:03.105655
2023-03-26 15:35:05.111016
2023-03-26 15:35:07.937911
2023-03-26 15:35:10.493186
2023-03-26 15:35:12.681495
2023-03-26 15:35:14.629409
2023-03-26 15:35:17.205042
2023-03-26 15:35:19.099010
2023-03-26 15:35:22.162769
2023-03-26 15:35:23.962530
2023-03-26 15:35:25.519422
2023-03-26 15:35:27.780112
2023-03-26 15:35:30.513598
2023-03-26 15:35:33.066231
2023-03-26 15:35:34.952196
2023-03-26 15:35:37.197053
2023-03-26 15:35:39.633494
2023-03-26 15:35:41.581091
2023-03-26 15:35:43.085923
2023-03-26 15:35:46.377031
2023-03-26 15:35:48.063549
2023-03-26 15:35:50.280304
2023-03-26 15:35:52.772199
2023-03-26 15:35:55.103593
2

In [73]:
write_to_db_sections(0, '/b/')

In [83]:
for idn in b_links:
    link = 'https://2ch.hk' + idn
    try:
        response = session.get(link, headers=headers, proxies=proxy)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        write_error(e)
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.get_text() != '':
        data = (pars_get_id(soup), pars_get_text(soup), pars_get_title(soup), pars_get_date(soup), link)
        write_to_db_data(data)
        write_to_db_conn_section(0, data[0])

    print(datetime.now())

2023-03-26 17:25:12.562911
2023-03-26 17:25:12.836609
2023-03-26 17:25:13.053068
2023-03-26 17:25:13.321167
2023-03-26 17:25:13.568644
2023-03-26 17:25:13.907277
2023-03-26 17:25:14.271382
2023-03-26 17:25:14.474605
2023-03-26 17:25:16.043973
2023-03-26 17:25:16.399358
2023-03-26 17:25:16.587215
2023-03-26 17:25:16.703163
2023-03-26 17:25:17.176169
2023-03-26 17:25:17.300187
2023-03-26 17:25:17.527685
2023-03-26 17:25:17.636116
2023-03-26 17:25:17.829220
2023-03-26 17:25:17.951166
2023-03-26 17:25:18.567162
2023-03-26 17:25:19.552059
2023-03-26 17:25:20.270592
2023-03-26 17:25:20.575012
2023-03-26 17:25:21.626238
2023-03-26 17:25:21.768443
2023-03-26 17:25:22.473428
2023-03-26 17:25:22.640741
2023-03-26 17:25:22.777767
2023-03-26 17:25:22.902720
2023-03-26 17:25:23.126762
2023-03-26 17:25:23.367190
2023-03-26 17:25:23.591167
2023-03-26 17:25:25.200259
2023-03-26 17:25:26.443664
2023-03-26 17:25:26.635193
2023-03-26 17:25:26.844896
2023-03-26 17:25:27.018495
2023-03-26 17:25:27.174497
2

2023-03-26 17:27:33.670398
2023-03-26 17:27:34.179841
2023-03-26 17:27:34.587664
2023-03-26 17:27:34.777760
2023-03-26 17:27:35.276879
2023-03-26 17:27:35.492082
2023-03-26 17:27:35.609356
2023-03-26 17:27:37.302924
2023-03-26 17:27:37.727118
2023-03-26 17:27:38.003766
2023-03-26 17:27:38.193509
2023-03-26 17:27:38.323922
2023-03-26 17:27:38.435984
2023-03-26 17:27:38.553396
2023-03-26 17:27:38.858262
2023-03-26 17:27:39.063221
2023-03-26 17:27:39.667611
2023-03-26 17:27:39.797735
2023-03-26 17:27:39.978424
2023-03-26 17:27:41.388388
2023-03-26 17:27:41.560237
2023-03-26 17:27:41.697494
2023-03-26 17:27:43.339537
2023-03-26 17:27:44.198644
2023-03-26 17:27:44.392748
2023-03-26 17:27:44.601251
2023-03-26 17:27:44.778861
2023-03-26 17:27:45.766917
2023-03-26 17:27:45.954597
2023-03-26 17:27:46.138652
2023-03-26 17:27:47.444023
2023-03-26 17:27:47.642429
2023-03-26 17:27:47.813682
2023-03-26 17:27:47.960785
2023-03-26 17:27:48.177097
2023-03-26 17:27:49.408703
2023-03-26 17:27:49.556591
2

2023-03-26 17:29:41.076087
2023-03-26 17:29:41.249790
2023-03-26 17:29:41.579534
2023-03-26 17:29:41.802159
2023-03-26 17:29:41.991331
2023-03-26 17:29:42.154211
2023-03-26 17:29:42.406533
2023-03-26 17:29:44.004197
2023-03-26 17:29:44.256748
2023-03-26 17:29:44.474019
2023-03-26 17:29:45.066120
2023-03-26 17:29:45.219743
2023-03-26 17:29:45.429556
2023-03-26 17:29:45.753772
2023-03-26 17:29:46.052048
2023-03-26 17:29:46.693581
2023-03-26 17:29:47.154990
2023-03-26 17:29:47.486948
2023-03-26 17:29:49.114232
2023-03-26 17:29:49.419880
2023-03-26 17:29:49.832785
2023-03-26 17:29:50.045712
2023-03-26 17:29:50.918709
2023-03-26 17:29:51.253837
2023-03-26 17:29:52.047501
2023-03-26 17:29:52.306237
2023-03-26 17:29:52.539544
2023-03-26 17:29:52.776699
2023-03-26 17:29:52.965687
2023-03-26 17:29:54.640865
2023-03-26 17:29:54.825581
2023-03-26 17:29:55.041950
2023-03-26 17:29:55.560010
2023-03-26 17:29:55.869772
2023-03-26 17:29:56.633913
2023-03-26 17:29:56.842480
2023-03-26 17:29:58.282257
2

2023-03-26 17:32:03.448585
2023-03-26 17:32:03.659103
2023-03-26 17:32:03.943735
2023-03-26 17:32:04.061255
2023-03-26 17:32:05.738490
2023-03-26 17:32:05.957294
2023-03-26 17:32:06.156894
2023-03-26 17:32:06.370573
2023-03-26 17:32:06.546473
2023-03-26 17:32:06.721232
2023-03-26 17:32:07.015049
2023-03-26 17:32:07.261614
2023-03-26 17:32:07.469497
2023-03-26 17:32:07.679060
2023-03-26 17:32:07.903917
2023-03-26 17:32:08.152061
2023-03-26 17:32:08.824731
2023-03-26 17:32:10.323535
2023-03-26 17:32:10.538235
2023-03-26 17:32:10.803175
2023-03-26 17:32:11.052043
2023-03-26 17:32:11.905043
2023-03-26 17:32:12.213158
2023-03-26 17:32:12.451356
2023-03-26 17:32:12.735687
2023-03-26 17:32:14.615144
2023-03-26 17:32:14.820606
2023-03-26 17:32:15.029888
2023-03-26 17:32:15.698785
2023-03-26 17:32:16.122067
2023-03-26 17:32:16.415712
2023-03-26 17:32:16.726527
2023-03-26 17:32:16.979502
2023-03-26 17:32:17.180335
2023-03-26 17:32:17.607289
2023-03-26 17:32:17.808572
2023-03-26 17:32:18.095974
2

2023-03-26 17:34:31.666684
2023-03-26 17:34:31.857186
2023-03-26 17:34:33.700164
2023-03-26 17:34:33.932354
2023-03-26 17:34:34.149312
2023-03-26 17:34:34.446957
2023-03-26 17:34:35.904313
2023-03-26 17:34:37.219282
2023-03-26 17:34:38.538040
2023-03-26 17:34:38.763592
2023-03-26 17:34:38.978995
2023-03-26 17:34:39.504983
2023-03-26 17:34:39.686802
2023-03-26 17:34:39.849663
2023-03-26 17:34:40.185737
2023-03-26 17:34:40.451794
2023-03-26 17:34:40.619809
2023-03-26 17:34:40.813393
2023-03-26 17:34:40.942387
2023-03-26 17:34:42.506397
2023-03-26 17:34:42.635936
2023-03-26 17:34:42.857309
2023-03-26 17:34:44.158687
2023-03-26 17:34:44.364407
2023-03-26 17:34:44.560445
2023-03-26 17:34:44.853254
2023-03-26 17:34:46.257661
2023-03-26 17:34:47.907039
2023-03-26 17:34:49.275576
2023-03-26 17:34:49.493722
2023-03-26 17:34:50.780949
2023-03-26 17:34:51.105129
2023-03-26 17:34:51.390232
2023-03-26 17:34:51.595473
2023-03-26 17:34:51.900972
2023-03-26 17:34:52.141540
2023-03-26 17:34:52.381425
2

...

2023-03-26 17:43:43.867731
2023-03-26 17:43:44.111650
2023-03-26 17:43:44.623296
2023-03-26 17:43:45.030180
2023-03-26 17:43:45.462388
2023-03-26 17:43:47.216675
2023-03-26 17:43:47.607124
2023-03-26 17:43:47.841175
2023-03-26 17:43:48.700049
2023-03-26 17:43:48.990615
2023-03-26 17:43:49.221607
2023-03-26 17:43:49.434906
2023-03-26 17:43:49.830588
2023-03-26 17:43:50.094245
2023-03-26 17:43:50.794201
2023-03-26 17:43:51.951991
2023-03-26 17:43:52.120470
2023-03-26 17:43:53.497472
2023-03-26 17:43:53.723659
2023-03-26 17:43:55.568718
2023-03-26 17:43:57.553877
2023-03-26 17:43:58.137627
2023-03-26 17:43:59.513290
2023-03-26 17:43:59.718796
2023-03-26 17:44:00.174636
2023-03-26 17:44:00.580378
2023-03-26 17:44:00.808731
2023-03-26 17:44:01.015825
2023-03-26 17:44:01.197570
2023-03-26 17:44:01.417568
2023-03-26 17:44:01.806776
2023-03-26 17:44:02.064512
2023-03-26 17:44:02.291539
2023-03-26 17:44:03.192905
2023-03-26 17:44:03.590175
2023-03-26 17:44:03.985907
2023-03-26 17:44:04.186110
2

2023-03-26 17:46:23.862298
2023-03-26 17:46:24.124106
2023-03-26 17:46:24.340454
2023-03-26 17:46:24.649478
2023-03-26 17:46:25.025462
2023-03-26 17:46:25.259792
2023-03-26 17:46:25.472567
2023-03-26 17:46:26.338112
2023-03-26 17:46:26.595361
2023-03-26 17:46:26.818364
2023-03-26 17:46:27.061912
2023-03-26 17:46:28.344615
2023-03-26 17:46:28.467271
2023-03-26 17:46:28.961869
2023-03-26 17:46:29.102394
2023-03-26 17:46:29.404237
2023-03-26 17:46:30.875186
2023-03-26 17:46:32.008795
2023-03-26 17:46:32.252750
2023-03-26 17:46:32.444784
2023-03-26 17:46:32.664043
2023-03-26 17:46:34.264173
2023-03-26 17:46:35.417794
2023-03-26 17:46:35.614580
2023-03-26 17:46:35.923051
2023-03-26 17:46:36.153490
2023-03-26 17:46:36.356943
2023-03-26 17:46:36.578943
2023-03-26 17:46:37.607026
2023-03-26 17:46:38.251125
2023-03-26 17:46:39.460234
2023-03-26 17:46:39.663043
2023-03-26 17:46:40.031811
2023-03-26 17:46:40.290888
2023-03-26 17:46:41.087418
2023-03-26 17:46:41.253899
2023-03-26 17:46:41.609999
2

2023-03-26 17:48:53.768181
2023-03-26 17:48:55.147341
2023-03-26 17:48:55.398388
2023-03-26 17:48:55.899328
2023-03-26 17:48:56.076833
2023-03-26 17:48:58.076730
2023-03-26 17:48:58.293931
2023-03-26 17:48:58.486435
2023-03-26 17:48:58.664431
2023-03-26 17:48:58.901013
2023-03-26 17:48:59.131369
2023-03-26 17:48:59.357317
2023-03-26 17:48:59.663245
2023-03-26 17:49:00.301612
2023-03-26 17:49:00.502219
2023-03-26 17:49:02.099300
2023-03-26 17:49:02.305199
2023-03-26 17:49:02.490331
2023-03-26 17:49:02.714941
2023-03-26 17:49:03.086257
2023-03-26 17:49:03.343293
2023-03-26 17:49:03.563525
2023-03-26 17:49:03.983266
2023-03-26 17:49:04.209961
2023-03-26 17:49:04.616317
2023-03-26 17:49:04.801315
2023-03-26 17:49:06.576365
2023-03-26 17:49:06.834595
2023-03-26 17:49:07.125312
2023-03-26 17:49:07.509512
2023-03-26 17:49:09.054538
2023-03-26 17:49:09.312126
2023-03-26 17:49:09.624101
2023-03-26 17:49:11.216569
2023-03-26 17:49:11.591863
2023-03-26 17:49:11.954132
2023-03-26 17:49:12.718348
2

2023-03-26 17:51:37.075535
2023-03-26 17:51:37.439707
2023-03-26 17:51:38.083595
2023-03-26 17:51:38.391863
2023-03-26 17:51:38.678299
2023-03-26 17:51:38.892494
2023-03-26 17:51:39.351116
2023-03-26 17:51:40.800547
2023-03-26 17:51:41.102076
2023-03-26 17:51:41.496676
2023-03-26 17:51:42.975937
2023-03-26 17:51:43.453752
2023-03-26 17:51:44.089839
2023-03-26 17:51:44.868453
2023-03-26 17:51:45.667899
2023-03-26 17:51:45.987026
2023-03-26 17:51:46.364122
2023-03-26 17:51:46.604866
2023-03-26 17:51:48.094205
2023-03-26 17:51:48.396814
2023-03-26 17:51:48.603947
2023-03-26 17:51:48.864237
2023-03-26 17:51:49.244305
2023-03-26 17:51:49.666327
2023-03-26 17:51:50.004355
2023-03-26 17:51:50.223488
2023-03-26 17:51:50.546295
2023-03-26 17:51:50.858537
2023-03-26 17:51:52.316081
2023-03-26 17:51:52.563370
2023-03-26 17:51:53.621204
2023-03-26 17:51:54.720386
2023-03-26 17:51:55.594887
2023-03-26 17:51:55.926617
2023-03-26 17:51:56.358981
2023-03-26 17:51:56.690222
2023-03-26 17:51:57.138354
2

KeyboardInterrupt: 

In [85]:
news_links = []

In [84]:
write_to_db_sections(1, '/news/')

In [86]:
for idn in range(0, 101):
    try:
        response = session.get('https://2ch.hk/news/arch/' + str(idn) + '.html', headers=headers, proxies=proxy)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        write_error(e)
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    post = soup.find_all('a', href=True)
    for i in post:
        if i.get_text() not in falselinks:
            news_links.append(i['href'])
    
    print(datetime.now())

2023-03-26 17:56:36.387415
2023-03-26 17:56:36.646447
2023-03-26 17:56:37.234918
2023-03-26 17:56:37.626022
2023-03-26 17:56:38.044164
2023-03-26 17:56:38.415224
2023-03-26 17:56:38.782809
2023-03-26 17:56:39.206678
2023-03-26 17:56:39.758330
2023-03-26 17:56:40.152776
2023-03-26 17:56:40.460473
2023-03-26 17:56:40.827445
2023-03-26 17:56:41.149422
2023-03-26 17:56:41.475847
2023-03-26 17:56:41.914359
2023-03-26 17:56:42.318023
2023-03-26 17:56:42.600396
2023-03-26 17:56:42.993006
2023-03-26 17:56:43.364756
2023-03-26 17:56:43.741025
2023-03-26 17:56:44.206238
2023-03-26 17:56:44.525781
2023-03-26 17:56:44.836067
2023-03-26 17:56:45.111086
2023-03-26 17:56:45.391387
2023-03-26 17:56:46.004918
2023-03-26 17:56:46.395479
2023-03-26 17:56:46.661354
2023-03-26 17:56:46.951229
2023-03-26 17:56:47.236420
2023-03-26 17:56:47.507968
2023-03-26 17:56:47.807635
2023-03-26 17:56:48.230694
2023-03-26 17:56:48.762828
2023-03-26 17:56:49.174403
2023-03-26 17:56:49.545815
2023-03-26 17:56:49.892256
2

In [88]:
for idn in news_links:
    link = 'https://2ch.hk' + idn
    try:
        response = session.get(link, headers=headers, proxies=proxy)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        write_error(e)
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.get_text() != '':
        data = (pars_get_id(soup), pars_get_text(soup), pars_get_title(soup), pars_get_date(soup), link)
        write_to_db_data(data)
        write_to_db_conn_section(1, data[0])

    print(datetime.now())

2023-03-26 18:00:15.609805
2023-03-26 18:00:15.919536
2023-03-26 18:00:16.541465
2023-03-26 18:00:16.908748
2023-03-26 18:00:17.840462
2023-03-26 18:00:18.108812
2023-03-26 18:00:18.455819
2023-03-26 18:00:19.062847
2023-03-26 18:00:19.629355
2023-03-26 18:00:19.895852
2023-03-26 18:00:20.571601
2023-03-26 18:00:20.980623
2023-03-26 18:00:21.327966
2023-03-26 18:00:21.587396
2023-03-26 18:00:21.862664
2023-03-26 18:00:22.226746
2023-03-26 18:00:22.641405
2023-03-26 18:00:23.028366
2023-03-26 18:00:23.427256
2023-03-26 18:00:23.868267
2023-03-26 18:00:24.412451
2023-03-26 18:00:24.655262
2023-03-26 18:00:24.945796
2023-03-26 18:00:25.213507
2023-03-26 18:00:25.619674
2023-03-26 18:00:25.852475
2023-03-26 18:00:26.239035
2023-03-26 18:00:26.582218
2023-03-26 18:00:27.013987
2023-03-26 18:00:27.399137
2023-03-26 18:00:27.747233
2023-03-26 18:00:28.028093
2023-03-26 18:00:28.507249
2023-03-26 18:00:28.870943
2023-03-26 18:00:29.202099
2023-03-26 18:00:29.512228
2023-03-26 18:00:29.896754
2

2023-03-26 18:02:46.495671
2023-03-26 18:02:47.222647
2023-03-26 18:02:47.522224
2023-03-26 18:02:48.019642
2023-03-26 18:02:48.813597
2023-03-26 18:02:49.105411
2023-03-26 18:02:49.616192
2023-03-26 18:02:50.009693
2023-03-26 18:02:50.303391
2023-03-26 18:02:50.678709
2023-03-26 18:02:51.143447
2023-03-26 18:02:51.583863
2023-03-26 18:02:52.052999
2023-03-26 18:02:52.652363
2023-03-26 18:02:52.979385
2023-03-26 18:02:53.511552
2023-03-26 18:02:53.948497
2023-03-26 18:02:54.243822
2023-03-26 18:02:54.812270
2023-03-26 18:02:55.204165
2023-03-26 18:02:55.860305
2023-03-26 18:02:56.403553
2023-03-26 18:02:56.686542
2023-03-26 18:02:57.242564
2023-03-26 18:02:57.548411
2023-03-26 18:02:57.945455
2023-03-26 18:02:58.289611
2023-03-26 18:02:58.738269
2023-03-26 18:02:59.317443
2023-03-26 18:02:59.680391
2023-03-26 18:02:59.994680
2023-03-26 18:03:00.387242
2023-03-26 18:03:00.762426
2023-03-26 18:03:01.171693
2023-03-26 18:03:01.739448
2023-03-26 18:03:02.164435
2023-03-26 18:03:02.483969
2

2023-03-26 18:05:01.529232
2023-03-26 18:05:01.788990
2023-03-26 18:05:02.245411
2023-03-26 18:05:02.415501
2023-03-26 18:05:02.700729
2023-03-26 18:05:02.984319
2023-03-26 18:05:03.262351
2023-03-26 18:05:03.522633
2023-03-26 18:05:03.878937
2023-03-26 18:05:04.326011
2023-03-26 18:05:04.662014
2023-03-26 18:05:05.342290
2023-03-26 18:05:05.705653
2023-03-26 18:05:06.035460
2023-03-26 18:05:06.857373
2023-03-26 18:05:07.383833
2023-03-26 18:05:08.160163
2023-03-26 18:05:08.418776
2023-03-26 18:05:08.695606
2023-03-26 18:05:09.018830
2023-03-26 18:05:09.330861
2023-03-26 18:05:09.707752
2023-03-26 18:05:10.061873
2023-03-26 18:05:10.405645
2023-03-26 18:05:10.920736
2023-03-26 18:05:11.290743
2023-03-26 18:05:11.461493
2023-03-26 18:05:12.675687
2023-03-26 18:05:12.914541
2023-03-26 18:05:13.148879
2023-03-26 18:05:13.475706
2023-03-26 18:05:14.146267
2023-03-26 18:05:14.336766
2023-03-26 18:05:14.532608
2023-03-26 18:05:15.297119
2023-03-26 18:05:15.900939
2023-03-26 18:05:16.389206
2

2023-03-26 18:06:56.195822
2023-03-26 18:06:56.999322
2023-03-26 18:06:57.358165
2023-03-26 18:06:57.872316
2023-03-26 18:06:58.473671
2023-03-26 18:06:58.917261
2023-03-26 18:06:59.560781
2023-03-26 18:07:00.222562
2023-03-26 18:07:01.506125
2023-03-26 18:07:01.718098
2023-03-26 18:07:01.955016
2023-03-26 18:07:02.566662
2023-03-26 18:07:02.841544
2023-03-26 18:07:03.126647
2023-03-26 18:07:03.523033
2023-03-26 18:07:03.766636
2023-03-26 18:07:05.482974
2023-03-26 18:07:05.800816
2023-03-26 18:07:06.078377
2023-03-26 18:07:06.352473
2023-03-26 18:07:06.804122
2023-03-26 18:07:07.240419
2023-03-26 18:07:07.454588
2023-03-26 18:07:07.828512
2023-03-26 18:07:08.352340
2023-03-26 18:07:08.671140
2023-03-26 18:07:08.956844
2023-03-26 18:07:09.175207
2023-03-26 18:07:09.443419
2023-03-26 18:07:09.803481
2023-03-26 18:07:10.256899
2023-03-26 18:07:10.516972
2023-03-26 18:07:10.765489
2023-03-26 18:07:11.139521
2023-03-26 18:07:11.490378
2023-03-26 18:07:11.884156
2023-03-26 18:07:12.456768
2

2023-03-26 18:08:46.333397
2023-03-26 18:08:46.564792
2023-03-26 18:08:46.982304
2023-03-26 18:08:47.383876
2023-03-26 18:08:47.672427
2023-03-26 18:08:48.085205
2023-03-26 18:08:48.408810
2023-03-26 18:08:48.938339
2023-03-26 18:08:49.466667
2023-03-26 18:08:49.868310
2023-03-26 18:08:50.087510
2023-03-26 18:08:50.344059
2023-03-26 18:08:50.968612
2023-03-26 18:08:51.242649
2023-03-26 18:08:51.628272
2023-03-26 18:08:51.926020
2023-03-26 18:08:52.535686
2023-03-26 18:08:52.776534
2023-03-26 18:08:53.181683
2023-03-26 18:08:53.503004
2023-03-26 18:08:53.863560
2023-03-26 18:08:54.305467
2023-03-26 18:08:54.834434
2023-03-26 18:08:55.268753
2023-03-26 18:08:55.946715
2023-03-26 18:08:56.189234
2023-03-26 18:08:56.396043
2023-03-26 18:08:56.664877
2023-03-26 18:08:56.886403
2023-03-26 18:08:57.203899
2023-03-26 18:08:57.487712
2023-03-26 18:08:57.736927
2023-03-26 18:08:58.015377
2023-03-26 18:08:58.372332
2023-03-26 18:08:59.252008
2023-03-26 18:08:59.658064
2023-03-26 18:09:00.226766
2

...

2023-03-26 18:23:53.129984
2023-03-26 18:23:53.423535
2023-03-26 18:23:53.676544
2023-03-26 18:23:53.953048
2023-03-26 18:23:54.163374
2023-03-26 18:23:54.555063
2023-03-26 18:23:54.786607
2023-03-26 18:23:55.413291
2023-03-26 18:23:55.800147
2023-03-26 18:23:56.020640
2023-03-26 18:23:56.274008
2023-03-26 18:23:56.948836
2023-03-26 18:23:57.168421
2023-03-26 18:23:57.381550
2023-03-26 18:23:57.728730
2023-03-26 18:23:58.242184
2023-03-26 18:23:58.451346
2023-03-26 18:23:59.293061
2023-03-26 18:23:59.548049
2023-03-26 18:23:59.980120
2023-03-26 18:24:00.189797
2023-03-26 18:24:00.392511
2023-03-26 18:24:00.596361
2023-03-26 18:24:00.864261
2023-03-26 18:24:01.485175
2023-03-26 18:24:01.884677
2023-03-26 18:24:02.171686
2023-03-26 18:24:02.433173
2023-03-26 18:24:03.013554
2023-03-26 18:24:03.423380
2023-03-26 18:24:03.723416
2023-03-26 18:24:03.953668
2023-03-26 18:24:04.361168
2023-03-26 18:24:04.878416
2023-03-26 18:24:05.125882
2023-03-26 18:24:05.426829
2023-03-26 18:24:05.993194
2

2023-03-26 18:25:41.728565
2023-03-26 18:25:41.910654
2023-03-26 18:25:42.140472
2023-03-26 18:25:42.557440
2023-03-26 18:25:42.771513
2023-03-26 18:25:43.055520
2023-03-26 18:25:43.434886
2023-03-26 18:25:43.936014
2023-03-26 18:25:44.180850
2023-03-26 18:25:44.392517
2023-03-26 18:25:44.838351
2023-03-26 18:25:45.262821
2023-03-26 18:25:45.569507
2023-03-26 18:25:45.796576
2023-03-26 18:25:46.125661
2023-03-26 18:25:46.353242
2023-03-26 18:25:46.592076
2023-03-26 18:25:46.847147
2023-03-26 18:25:47.519707
2023-03-26 18:25:47.859371
2023-03-26 18:25:48.090855
2023-03-26 18:25:48.438210
2023-03-26 18:25:48.713912
2023-03-26 18:25:49.001475
2023-03-26 18:25:49.364348
2023-03-26 18:25:50.279687
2023-03-26 18:25:50.572341
2023-03-26 18:25:51.019850
2023-03-26 18:25:51.355671
2023-03-26 18:25:51.578096
2023-03-26 18:25:51.851751
2023-03-26 18:25:52.142200
2023-03-26 18:25:52.731040
2023-03-26 18:25:53.031237
2023-03-26 18:25:53.322951
2023-03-26 18:25:53.772600
2023-03-26 18:25:54.022635
2

2023-03-26 18:27:44.477336
2023-03-26 18:27:44.807660
2023-03-26 18:27:45.272253
2023-03-26 18:27:45.507058
2023-03-26 18:27:46.004973
2023-03-26 18:27:46.403929
2023-03-26 18:27:46.629964
2023-03-26 18:27:46.931399
2023-03-26 18:27:47.147856
2023-03-26 18:27:47.764189
2023-03-26 18:27:48.009827
2023-03-26 18:27:48.369532
2023-03-26 18:27:48.592015
2023-03-26 18:27:48.858905
2023-03-26 18:27:49.127004
2023-03-26 18:27:49.495623
2023-03-26 18:27:49.898156
2023-03-26 18:27:50.538163
2023-03-26 18:27:51.106658
2023-03-26 18:27:51.430854
2023-03-26 18:27:52.189347
2023-03-26 18:27:52.583094
2023-03-26 18:27:52.807000
2023-03-26 18:27:53.037869
2023-03-26 18:27:53.460319
2023-03-26 18:27:53.739283
2023-03-26 18:27:54.102137
2023-03-26 18:27:54.633847
2023-03-26 18:27:55.038068
2023-03-26 18:27:55.279900
2023-03-26 18:27:55.503582
2023-03-26 18:27:55.770345
2023-03-26 18:27:58.257468
2023-03-26 18:27:58.655850
2023-03-26 18:27:59.041565
2023-03-26 18:27:59.515155
2023-03-26 18:27:59.682476
2

2023-03-26 18:29:36.505288
2023-03-26 18:29:36.907292
2023-03-26 18:29:37.439589
2023-03-26 18:29:37.832903
2023-03-26 18:29:38.191945
2023-03-26 18:29:38.643440
2023-03-26 18:29:39.123063
2023-03-26 18:29:39.446805
2023-03-26 18:29:39.851923
2023-03-26 18:29:40.196366
2023-03-26 18:29:40.511003
2023-03-26 18:29:40.904321
2023-03-26 18:29:41.383667
2023-03-26 18:29:41.876728
2023-03-26 18:29:42.329902
2023-03-26 18:29:42.779684
2023-03-26 18:29:43.122980
2023-03-26 18:29:43.445045
2023-03-26 18:29:43.799546
2023-03-26 18:29:44.456180
2023-03-26 18:29:44.816657
2023-03-26 18:29:45.271632
2023-03-26 18:29:45.708618
2023-03-26 18:29:46.164074
2023-03-26 18:29:46.660062
2023-03-26 18:29:47.101242
2023-03-26 18:29:47.643413
2023-03-26 18:29:48.100209
2023-03-26 18:29:48.628742
2023-03-26 18:29:49.065356
2023-03-26 18:29:49.441284
2023-03-26 18:29:49.863872
2023-03-26 18:29:50.522729
2023-03-26 18:29:50.831983
2023-03-26 18:29:51.229897
2023-03-26 18:29:51.648449
2023-03-26 18:29:52.009530
2

KeyboardInterrupt: 

In [89]:
threads_query = '''
SELECT * FROM threads
'''

In [90]:
df_threads = pd.read_sql_query(threads_query, con=conn)

In [91]:
df_threads = df_threads.set_index('id')
df_threads

,text,name,date,link
id,,,,
70852,\nНемецкий слесарь Артур Фишер состоял в гитле...,/news/ - Билл Гейтс в мире ремонта. Умер Артур...,09/02/16 Втр 18:35:07,https://2ch.hk/news/arch/2016-02-09/res/70852....
71807,"\nВ Кирове автомобиль «Лада Калина», который п...",/news/ - В Кирове эвакуаторщик уронил «Ладу Ка...,10/02/16 Срд 18:27:51,https://2ch.hk/news/arch/2016-02-10/res/71807....
75112,\nГоссекретарь США Джон Керри выдвинул условия...,/news/ - Санкции никогда не закончатся сами по...,13/02/16 Суб 23:07:26,https://2ch.hk/news/arch/2016-02-13/res/75112....
78025,\nГруппа ученых из США сообщила о разработке р...,/news/ - Ученые из США сообщили о создании рев...,16/02/16 Втр 15:39:13,https://2ch.hk/news/arch/2016-02-16/res/78025....
78402,\nhttp://www.dualshockers.com/2016/02/15/un-co...,/news/ - ООН призвал Японию запретить игры и м...,16/02/16 Втр 22:25:09,https://2ch.hk/news/arch/2016-02-16/res/78402....
...,...,...,...,...
120050795,\nИМОУТО ТРЕДСестра 12-летняя начиталась мемов...,/b/ - ИМОУТО ТРЕД Сестра 12-летняя,15/03/16 Втр 00:08:08,https://2ch.hk/b/arch/2016-03-15/res/120050795...
120050880,"\nАнон, привет.Что делать, если завтра батя/ма...","/b/ - Анон, привет. Что делать,",15/03/16 Втр 00:08:56,https://2ch.hk/b/arch/2016-03-15/res/120050880...
120050964,"\nСап аноны, короче ситуация такая : вот вот (...","/b/ - Сап аноны, короче ситуация",15/03/16 Втр 00:09:49,https://2ch.hk/b/arch/2016-03-15/res/120050964...


In [92]:
threads_all = ''
for i in df_threads['text']:
    threads_all += i + 'separate_this'

In [93]:
mystem = Mystem()

In [ ]:
threads_lemmas = mystem.lemmatize(threads_all)

In [ ]:
threads_lemmas_text = ''.join(threads_lemmas)

In [ ]:
threads_lemmatized = threads_lemmas_text.split('separate_this')

In [ ]:
df_threads['lemmatized'] = threads_lemmatized

In [ ]:
df_threads

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
    ngram_range = (1,3),
    min_df = 3,
    stop_words = stopwords.words('russian'),
    token_pattern=r'(?u)\b(?!\d)\w\w+\b'
    )

In [ ]:
tfidf_vectorizer.fit(df_threads['lemmatized'])
vekt_threads = tfidf_vectorizer.transform(df_threads['lemmatized'])

vekt_threads_visual = pd.DataFrame(vekt_threads.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

vekt_threads_visual = vekt_threads_visual.set_index(df_threads.index)

vekt_threads_visual

In [ ]:
KeyWords = vekt_threads_visual.idxmax(axis=1)

KeyWords

In [ ]:
cur.execute("""
ALTER TABLE threads
    ADD keyword text
""")
conn.commit()

In [ ]:
for i in KeyWords.index:
    cur.execute('''UPDATE threads SET keyword = ? WHERE id = ?''', (KeyWords[i], i))
conn.commit()

In [ ]:
df_threads = pd.read_sql_query(threads_query, con=conn)

In [ ]:
df_threads = df_threads.set_index('id')
df_threads